### Genetic algorithms

We'll use GA to solve the onemax problem.

In [1]:
import numpy as np
from random import sample

In [2]:
def init_pop(pop_size,genome_len):
    ''' accepts: pop_size = population size
                 genome_len = the length of each genome
        returns: a list of critter genomes
    '''
    return([list(np.random.randint(0,2,genome_len)) for i in range(pop_size)])

In [ ]:
# check out the population

In [6]:
# we'll use onemax for our fitness
def fitness(critter):
    ''' accepts: critter = a critter (with a genome)
        returns: the fitness of the critter
    '''
    return(sum(critter))

In [37]:
def tourn(pop,tourn_size):
    ''' accepts: pop = a population
                 tourn_size = number of critters per tournament
        returns: the index of tournament winner
    '''
    selection = sample(range(len(pop)),tourn_size)
    fittest = selection[0]
    for idx in selection[1:]:
        if fitness(pop[idx])>fitness(pop[fittest]):
            fittest = idx
    return(fittest)

In [ ]:
# convince ourselves tournament works as it should

There are many methods of breeding, including asexual. The standard approach takes two tournament winners, randomly chooses a position in which to split the genomes, then splices the four pieces into two new genomes (offspring). The offspring are mutated then added to the next generation. Here's an example, up to mutation:

In [ ]:
'''
parent1 = 010010111
parent2 = 100110100
split_position: 3
parent1 stems = 010, 010111
parent2 stems = 100, 110100
offspring1 = 010+110100 = 010110100
offspring2 = 100+010111 = 100010111
'''

Mutation then randomly changes some bits in each offspring. The standard approach is to flip each bit with (independent) probability $p$, with the standard value of $p$ being the reciprocal of the genome length. So how many bits on average will be flipped?

Also, discuss why we mutate.

There is also standardly some probability of just copying the parents over to the new generation unchanged (bypassing the splice and mutation).

In [38]:
def breed(pop,idx1,idx2,mut_prob,copyover_prob):
    ''' accepts: pop = a population
                 idx1 = index of first critter to breed
                 idx2 = index of second critter to breed
                 mut_prob = in mutation, probability each bit is flipped
                 copyover_prob = probility parents copy unchanged into next gen
        returns: [offspring1, offspring2] = two offspring (with mutation)
    '''
    if np.random.uniform(0,1) < copyover_prob:
        return(pop[idx1],pop[idx2])
    genome_len = len(pop[0]) # get length of each genome
    split_pos = np.random.randint(1,genome_len) # where to split the genomes
    offspring1 = pop[idx1][:split_pos]+pop[idx2][split_pos:]
    offspring2 = pop[idx2][:split_pos]+pop[idx1][split_pos:]
    # mutate the offspring
    for critter in [offspring1,offspring2]:
        for i in range(len(critter)):
            if np.random.uniform(0,1) < mut_prob:
                critter[i] = 1-critter[i]
    return(offspring1,offspring2)

# again, one should investigate this to make sure it works as intended

In [39]:
# create new generation function
def next_gen(pop,tourn_size,mut_prob,copyover_prob):
    next_gen = []
    while len(next_gen)<len(pop)-2: # save two spots for elitism
        winner1 = tourn(pop,tourn_size)
        winner2 = tourn(pop,tourn_size)
        next_gen += breed(pop,winner1,winner2,mut_prob,copyover_prob)
    # implement elitism
    pop.sort(key = lambda crit:fitness(crit),reverse = True)
    next_gen += pop[:2]
    return(next_gen)

In [40]:
# finds the best genome and fitness of a population
def best(pop):
    best_genome = pop[0]
    best_fitness = fitness(pop[0])
    for crit in pop[1:]:
        if fitness(crit) > best_fitness:
            best_fitness = fitness(crit)
            best_genome = crit
    return(best_genome,best_fitness)

Now we need to put it together. We define the constants, then iterate through generations. We track the best solution so far by checking the fittest individual in each generation. Elitism is a standard technique which automatically (no tournament) adds the fittest several individuals to the next generation.

We must include a quit condition. In this case, since we know the optimum fitness, we can just define the quit condition to be when a solution attains optimum fitness. Of course, for general applications, we will not know the optimum fitness, so this is sort of cheating. A typical quit condition will be when a maximum number of generations is reached or when the fittest individual has not improved in some number of generations, whichever comes first.

In [45]:
max_generations = 1000 # max number of generations before quit
max_improvement_counter = 25 # max number of generations without improvement before quit
pop_size = 20
genome_len = 100
tourn_size = 5
mut_prob = 1/genome_len
copyover_prob = 0.1

In [46]:
generation_count = 0 # counts how many generations have elapsed
improvement_counter = 0 # counts how long since the last improvement in best_fitness

pop = init_pop(pop_size,genome_len)
# initialize best_fitness; it will be recomputed each generation
(best_genome, best_fitness) = best(pop)

while generation_count < max_generations:
    if generation_count>0:# in [10**k for k in range(10)]:
        print(f'gen: {generation_count}, best: {best_fitness}')
    improvement_counter += 1
    generation_count += 1
    pop = next_gen(pop,tourn_size,mut_prob,copyover_prob)
    (best_genome_current, best_fitness_current) = best(pop)
    if best_fitness_current > best_fitness:
        best_fitness = best_fitness_current
        best_genome = best_genome_current
        improvement_counter = 0
    if improvement_counter > max_improvement_counter or generation_count > max_generations:
        break

gen: 1, best: 59
gen: 2, best: 61
gen: 3, best: 62
gen: 4, best: 63
gen: 5, best: 64
gen: 6, best: 64
gen: 7, best: 66
gen: 8, best: 68
gen: 9, best: 68
gen: 10, best: 69
gen: 11, best: 72
gen: 12, best: 72
gen: 13, best: 72
gen: 14, best: 74
gen: 15, best: 76
gen: 16, best: 79
gen: 17, best: 79
gen: 18, best: 79
gen: 19, best: 80
gen: 20, best: 81
gen: 21, best: 82
gen: 22, best: 82
gen: 23, best: 83
gen: 24, best: 86
gen: 25, best: 87
gen: 26, best: 89
gen: 27, best: 89
gen: 28, best: 90
gen: 29, best: 90
gen: 30, best: 90
gen: 31, best: 91
gen: 32, best: 92
gen: 33, best: 92
gen: 34, best: 94
gen: 35, best: 94
gen: 36, best: 94
gen: 37, best: 94
gen: 38, best: 94
gen: 39, best: 95
gen: 40, best: 95
gen: 41, best: 95
gen: 42, best: 96
gen: 43, best: 97
gen: 44, best: 97
gen: 45, best: 97
gen: 46, best: 97
gen: 47, best: 97
gen: 48, best: 97
gen: 49, best: 97
gen: 50, best: 98
gen: 51, best: 98
gen: 52, best: 98
gen: 53, best: 98
gen: 54, best: 98
gen: 55, best: 98
gen: 56, best: 98
g